In [1]:
import model_utils as mu
mu.DEATH_RATE = 0.36
mu.ICU_RATE = 0.78
mu.HOSPITAL_RATE = 2.18
mu.SYMPTOM_RATE = 10.2
mu.INFECT_2_HOSPITAL_TIME = 11
mu.HOSPITAL_2_ICU_TIME = 4
mu.ICU_2_DEATH_TIME = 4
mu.ICU_2_RECOVER_TIME = 7
mu.NOT_ICU_DISCHARGE_TIME = 5

In [2]:
import pandas as pd
import numpy as np
import pwlf_mod as pwlf
import datetime as dt
import matplotlib.pyplot as plt

In [3]:
#Use plotly
import plotly.offline as py_offline
import cufflinks as cf
cf.go_offline()
py_offline.__PLOTLY_OFFLINE_INITIALIZED = True

In [4]:
import imp; imp.reload(mu)

<module 'model_utils' from '/home/quoc/github/Covid19_VN/model_utils.py'>

In [5]:
sheet_url = "https://docs.google.com/spreadsheets/d/1iviYd0Cha8K2bHkVTtSwyo9fxrusqaMfm6QP-Iw75Eg/edit#gid=0"

In [6]:
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

In [7]:
cases = pd.read_csv(url_1).set_index('date')

In [8]:
cases.index = pd.to_datetime(cases.index)

In [9]:
cases.iplot()

In [10]:
def get_transformed_data(data, smoothing_days=7):
    data_avg = data.rolling(smoothing_days, min_periods=3).mean()
    # Turn 0 to nan to avoid log of 0
    data_avg[data_avg <= 0.01] = np.nan
    # Because of this smoothing step, we need to time var of prediction by smoothing_days=3.
    # Rolling set the label at the right edge of the windows, so we need to blank out the first 7 days after
    # policy effective dates since it mixes before and after change curve
    #shift ahead 1 day to avoid overfitted due to average of exponential value
    #daily_local_death_new = daily_local_death_new.shift(1)
    data_avg_log = np.log(data_avg)
    return data_avg_log

In [11]:
get_transformed_data(cases).iplot(title ="Logarithm của trung bình 7 ngày")

In [12]:
log_critical = get_transformed_data(cases)

In [13]:
policy_change_dates =['2021-06-01']
policy_effective_dates = pd.to_datetime(policy_change_dates) + dt.timedelta(
        mu.INFECT_2_HOSPITAL_TIME)


In [14]:
def predict(data, forecast_horizon=30, policy_effective_dates=[], smoothing_days=7):
    '''Since this is highly contagious disease. Daily new death, which is a proxy for daily new infected cases
    is model as d(t)=a*d(t-1) or equivalent to d(t) = b*a^(t). After a log transform, it becomes linear.
    log(d(t))=logb+t*loga, so we can use linear regression to provide forecast (use robust linear regressor to avoid
    data anomaly in death reporting)
    There are two separate linear curves, one before the lockdown is effective(21 days after lockdown) and one after
    For using this prediction to infer back the other metrics (infected cases, hospital, ICU, etc..) only the before
    curve is used and valid. If we assume there is no new infection after lock down (perfect lockdown), the after
    curve only depends on the distribution of time to death since ICU.
    WARNING: if lockdown_date is not provided, we will default to no lockdown to raise awareness of worst case
    if no action. If you have info on lockdown date please use it to make sure the model provide accurate result'''
    #for policy_effective_date in policy_effective_dates:
    #    data = data.loc[
    #        (data.index > policy_effective_date + dt.timedelta(smoothing_days)) |
    #        (data.index <= policy_effective_date)]

    # log_daily_death.dropna(inplace=True)
    data_start_date = min(data.index)
    data_end_date = max(data.index)
    forecast_end_date = data_end_date + dt.timedelta(forecast_horizon)
    forecast_date_index = pd.date_range(start=data_start_date, end=forecast_end_date)

    data_start_date_idx = 0
    data_end_date_idx = (data_end_date - data_start_date).days
    forecast_end_date_idx = data_end_date_idx + forecast_horizon
    forecast_time_idx = (forecast_date_index - data_start_date).days.values
    data_time_idx = (data.index - data_start_date).days.values
    policy_effective_dates_idx = (policy_effective_dates - data_start_date).days.values
    data['time_idx'] = data_time_idx
    #import pdb; pdb.set_trace()
    data = data.replace([np.inf, -np.inf], np.nan).dropna()
    break_points = np.array([data_start_date_idx, ] +
                            policy_effective_dates_idx[(~np.isnan(policy_effective_dates_idx))&
                                                       (policy_effective_dates_idx < forecast_end_date_idx)].tolist() +
                            [forecast_end_date_idx, ])
    regr_pw = pwlf.PiecewiseLinFit(x=data.time_idx.values, y=data.f0_community)
    regr_pw.fit_with_breaks(break_points)
    model_beta = regr_pw.beta
    log_predicted_var = smoothing_days * regr_pw.prediction_variance(forecast_time_idx)

    # Use default slope when data is not enough to fit last line, less than 4 data point, with contain_rate=1 mean slope
    # is the same as previous slope (same policy) and 0 mean (relax 100%) slope will be same as before lockdown

    if ((data_end_date_idx-break_points[-2]) < 4) | (model_beta[-1] > max(0.3, abs(model_beta[1]))):
        if model_beta[-2] < 0:
            model_beta[-1] = (-model_beta[-2])*(1-contain_rate)
        else:
            model_beta[-1] = (-model_beta[-2])*(1+contain_rate)
        print("Use default last slope due to not enough data")
        #import pdb; pdb.set_trace()
        variance = log_predicted_var[sum(forecast_time_idx <= break_points[-2])]
        log_predicted_var_oos = variance * (forecast_time_idx[forecast_time_idx > break_points[-2]] -
                                                       break_points[-2])
        log_predicted_var = np.concatenate(
                (log_predicted_var[:sum(forecast_time_idx <= break_points[-2])],
                 log_predicted_var_oos))

    log_predicted_values = regr_pw.predict(forecast_time_idx, beta=model_beta, breaks=break_points)

    log_predicted_lower_bound_values = log_predicted_values - 1.96 * np.sqrt(log_predicted_var)
    log_predicted_upper_bound_values = log_predicted_values + 1.96 * np.sqrt(log_predicted_var)

    log_predicted = pd.DataFrame(log_predicted_values, index=forecast_date_index)
    log_predicted_lower_bound = pd.DataFrame(log_predicted_lower_bound_values, index=forecast_date_index)
    log_predicted_upper_bound = pd.DataFrame(log_predicted_upper_bound_values, index=forecast_date_index)
    log_predicted.columns = ['predicted']
    log_predicted_lower_bound.columns = ['lower_bound']
    log_predicted_upper_bound.columns = ['upper_bound']
    return log_predicted, log_predicted_lower_bound, log_predicted_upper_bound, regr_pw.beta


In [15]:
def get_infected_cases(data):
    '''This number only is close to number of confirmed case in country very early in the disease and 
    can still do contact tracing or very wide testing, eg. South Korea, Germany'''
    delay_time = mu.INFECT_2_HOSPITAL_TIME 
    infected_cases = (100/mu.SYMPTOM_RATE)*data.tshift(-delay_time)
    infected_cases.columns = ['infected']
    return infected_cases


In [16]:
log_predict_f0_community, lower, upper,_ = predict(log_critical[['f0_community']], policy_effective_dates=policy_effective_dates)

In [17]:
log_f0 = pd.concat([log_predict_f0_community, lower, upper, log_critical], axis=1, sort=True)

In [18]:
log_f0_ve = log_f0.rename(columns={'f0_community': 'Log(F0 tầm soát)', 'predicted': 'Dự báo Log(F0 tầm soát)',
                      'lower_bound': 'Cận Dưới', 'upper_bound': 'Cận Trên', 'f0':'Log(F0)'})\
.drop(columns=['ICU_current'], errors='ignore')

In [19]:
f0 = pd.concat([get_infected_cases(np.exp(log_predict_f0_community)), cases, np.exp(log_predict_f0_community), np.exp(lower), np.exp(upper), 
                ], axis=1, sort=True)

In [20]:
log_f0_ve.loc[:'2021-07-04'].iplot(title = "Biểu độ dự báo tình hình lây nhiễm theo Log scale")

In [21]:
f0_ve = f0.rename(columns={'f0_community': 'F0 tầm soát', 'predicted': 'Dự báo F0 tầm soát', 'infected': 'Dự báo F0',
                      'lower_bound': 'Cận Dưới F0 tầm soát', 'upper_bound': 'Cận Trên F0 tầm soát', 'f0':'F0'})\
.drop(columns=['ICU_current'], errors='ignore')

In [22]:
f0_ve.loc[:'2021-07-04'].iplot(title = "Biểu độ dự báo tình hình lây nhiễm")

# Tóm tắt

Số  ca lây nhiễm được tầm soát trong cộng đồng cho thấy mức độ lây nhiễm thực sự của Covid19 ở TP

Với cùng một chính sách giãn cách xã hội, đường logarithm của số ca lây nhiễm được tầm soát trong cộng đồng là đường thẳng, ta tạm gọi là "đường lây nhiễm". 

Đường lây nhiễm này thay đổi 11 ngày sau khi một chính sách mới có hiệu lực. 11 ngày là số ngày từ lúc bắt đầu nhiễm đến lúc có triệu chứng rõ rệt phải đi khám ở bệnh viện.

Sau khi TP thực hiện chỉ thị 15+ 16 (Gò Vấp, Q12) ngày 1/6, đường lây nhiễm có thay đổi kể từ ngày 12/6 nhưng chưa đi xuống

11 ngày sau khi thực hiện chỉ thị 10 của thành phố, khoảng 1/7, đường lây nhiễm này sẽ thay đổi.

Đường F0 (màu xanh dương) là thể hiện cả 3 nỗ lực cách ly, xét nghiệm và truy vết. Ở giai đoạn đầu, trước 12/6, số ca F0 được tìm thấy còn thấp hơn số ca F0 trong thực tế (màu cam) nên dịch tiếp tục lây lan tiềm ẩn trong cộng đồng. Sau ngày 12/6, nhờ nỗ lực xét nghiệm, truy vết và cách ly, đường xanh dương đã cao hơn đường cam. Nghĩa là ta đã tìm thấy thêm các ca trong các nhánh lây tiềm ẩn trước đây. Đây là cơ sở để ta có thể dập dịch hoàn toàn, khi đường dự báo F0 tầm soát (màu tím) tiến về 0. Điều này cho thấy chỉ thị 15+16 ngày 1/6 đã có hiệu quả, nhưng chưa đủ.  